### Gradio and HuggingFace

In this demo, we show how to build ready to deploy or use deep learning models. 

Hugging Face hosts thousands of pre-trained models in [Model Hub](https://huggingface.co/models). They also built high-level APIs so we can easily use and deploy pre-trained models using [Pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines). 

`gradio` provides APIs so we can easily build web applications that use our pre-trained models from Hugging Face. `gradio` also provides APIs so we can easily incorporate input and output web UIs.

After building the `gradio`, we can have permanent hosting using [Hugging Face Spaces](https://huggingface.co/blog/gradio-spaces). 

Let us first install Hugging Face `transformers` and `gradio`.

**Note:** For some examples, it is best to launch the app in another tab to enable access to the required inputs such as microphone or webcam. Running the app may also lock the `python` kernel and the notebook becomes unresponsive. In that case, please restart the kernel and clear the output. 

In [ ]:
!pip install transformers
!pip install gradio

#### Hello world in gradio

As a tradition, let us build the simplest `gradio` app. It accepts a `text` input and calls the `greet()` function to process this input and convert into another text. The output of `greet()` becomes the output of the `gradio` app.

To see our application, we call `launch()` after constructing our `gradio` `Interface`.

In [1]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!!"

gr.Interface(fn=greet, inputs="text", outputs="text").launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


#### Object Recognition using ResNet18

In our discussion about PyTorch, we used a pre-trained ResNet18 model from `torchvision`. We use `jupyter` notebook to show the results. The `jupyter` notebook is not an application that we can deploy and other people use with ease. The same with Google's colab. 

In this example, we use `gradio` to build a simple app that an end user can easily interact with. We reuse the code from our previous [example](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/pytorch_demo.ipynb).

In [9]:
import gradio as gr
import torch 
import torchvision
import torchvision.transforms as transforms
import requests
from einops import rearrange

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

resnet = torchvision.models.resnet18(pretrained=True)
resnet.eval()

# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

def classify(img):
    # By default, gradio image is numpy
    img = torch.from_numpy(img)
    # Numpy image is channel last. PyTorch is channel 1st.
    img = rearrange(img, 'h w c -> c h w')
    
    # The transforms before prediction
    img = torchvision.transforms.Resize(256)(img)
    img = torchvision.transforms.CenterCrop(224)(img).float()/255.
    img = normalize(img)
    
    # We insert batch size of 1
    img = rearrange(img, 'c h w  -> 1 c h w')
    
    # The actual prediction
    with torch.no_grad():
        pred = resnet(img)
    
    # Convert the prediction to probabilities
    pred = torch.nn.functional.softmax(pred, dim=1)
    # Remove the batch dim. torch.squeeze() can also be used.
    pred = rearrange(pred, "1 j->j")
    
    # torch to numpy space
    pred = pred.cpu().numpy()
    
    return {labels[i]: float(pred[i]) for i in range(1000)}
    

gr.Interface(fn=classify, 
             inputs=gr.inputs.Image(shape=(224, 224)),
             outputs=gr.outputs.Label(num_top_classes=5),
             title="1k Object Recognition",
             examples=['wonder_cat.jpg', 'aki_dog.jpg',],
             description="Demonstrates a pre-trained model from torchvision for image classification.",
             allow_flagging="never").launch(inbrowser=True)

/Users/rowel/anaconda3/envs/llm/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/rowel/anaconda3/envs/llm/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/rowel/anaconda3/envs/llm/lib/python3.10/site-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/Users/rowel/anaconda3/envs/llm/lib/python3.10/site-packages/gradio/deprecation.py:40: User

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


#### Using HuggingFace and Gradio

Loading a pre-trained model from torchvision, pre-processing the input, and post processing the output are all messy. Sometimes, we just want to load and use a machine learning model. Hugging Face provides a shortcut for all these steps through the use of `pipeline`. In `pipeline`, we supply the task name and the pre-trained model that is stored in Hugging Face Model Hub.

In this example, we use a much better model compared to ResNet18. It is called [BEIT](https://arxiv.org/abs/2106.08254) and can classify objects up to about 22k categories. We construct the `gradio` app by calling `from_pipeline()`.

In [10]:
import gradio as gr
from transformers import pipeline

pipe = pipeline(task="image-classification", 
                 # model that can do 22k-category classification
                 model="microsoft/beit-base-patch16-224-pt22k-ft22k")
gr.Interface.from_pipeline(pipe, 
                           title="22k Image Classification",
                           description="Object Recognition using Microsoft BEIT",
                           examples = ['wonder_cat.jpg', 'aki_dog.jpg',],
                           allow_flagging="never").launch(inbrowser=True)

/Users/rowel/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/models/beit/feature_extraction_beit.py:28: FutureWarning: The class BeitFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use BeitImageProcessor instead.
  warnings.warn(
/Users/rowel/anaconda3/envs/llm/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


#### Automatic Speech Recognition (ASR)

Let us shift to audio or speech domain. In this example, we demonstrate an Automatic Speech Recognition (ASR) system. We will use our microphone to record audio which is then converted to text using ASR. In this example, best to open the application in another browser tab by setting `inbrowser=True`.

Before running the `gradio` app, this ASR requires `sentencepice` module. Let us install it first.

In [4]:
!pip install sentencepiece

In this ASR, we use Facebook [S2T](https://arxiv.org/abs/2010.05171), a transformer-based speech to text model that is trained on librispeech dataset.

In [11]:
import gradio as gr
from transformers import pipeline

pipe = pipeline(task="automatic-speech-recognition", 
                model="facebook/s2t-medium-librispeech-asr")
gr.Interface.from_pipeline(pipe,
                           title="Automatic Speech Recognition (ASR)",
                           description="Using pipeline with Facebook S2T for ASR.",
                           examples=['data/ljspeech.wav',],
                           ).launch(inbrowser=True)

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


/Users/rowel/anaconda3/envs/llm/lib/python3.10/site-packages/gradio/processing_utils.py:242: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
/Users/rowel/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


#### Text to Speech (TTS)

Let us do the reverse of ASR or Text to Speech (TTS). In this example, we supply text and this text is converted to speech using the voice of Linda Johnson. We use a pre-trained model of [FastSpeech2](https://arxiv.org/abs/2006.04558) that is provided by Facebook in Model Hub.

In this example, we use [`load()`](https://gradio.app/docs/#load) method to load the pre-trained model. 

In [6]:
import gradio as gr
gr.Interface.load("huggingface/facebook/fastspeech2-en-ljspeech", 
                  description="TTS using FastSpeech2",
                  title="Text to Speech (TTS)",
                  examples=[["The quick brown fox jumps over the lazy dog."]]
                  ).launch()

Fetching model from: https://huggingface.co/facebook/fastspeech2-en-ljspeech
Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


#### Text Generation using GPT2

In this example, we use a large language model (LLM) by OpenAI to generate text. It is called [GPT2](https://github.com/openai/gpt-2). Text generation is one of the tasks where a language model can help us. Basically, we provide the initial text made of few words. Then, the model will continue it.

In [7]:
import gradio as gr
gr.Interface.load("huggingface/gpt2",
                  title="Text Generation",
                  description="Using GPT2.",
                  allow_flagging="never").launch(inbrowser=True)

Fetching model from: https://huggingface.co/gpt2
Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


#### Using `gr.Series()` to automatically chain the I/O of multiple models

In this example, we use the previous text generator as input to our TTS. We can for example use to generate a podcast on a certain topic. In this case, we replace GPT2 with a much better model called GPT Neo. 

In [8]:
import gradio as gr


textgen = gr.Interface.load("huggingface/EleutherAI/gpt-neo-2.7B")

tts = gr.Interface.load("huggingface/facebook/fastspeech2-en-ljspeech")

iface = gr.Series(textgen, tts)

iface.title = "Generated Text to Speech"
iface.allow_flagging = "never"
iface.launch(inbrowser=True)



Fetching model from: https://huggingface.co/EleutherAI/gpt-neo-2.7B
Fetching model from: https://huggingface.co/facebook/fastspeech2-en-ljspeech
Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
